# A time Story of offshore Societies

You can see the aim of this project in the README file.

Our work is based on 4 databases, namely the *bahamas leaks*, the *panama papers*, the *offshore leaks* and the *paradise papers*. They all contain csv files with all the data of a graph : nodes (one file for each type of node), and edges. We merged all these files in this notebook.

You will see all the steps of our data exploration. In the end our objective is to arrive to two Dataframe, one containing all the nodes and the relevant information, and one containing the edges with relevant information.

## Imports

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx #For graphs
import copy
import warnings
from datetime import *
import dateutil.parser
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

# https://www.occrp.org/en/panamapapers/database
# TRUMP OFFSHORE INC. is good example to see all entities interacting

## Filenames / paths

The data is separated for every leak source. For each leak source there is a folder containing the nodes of the graph, that can be of different types : <i>intermediary, officer, entity, address</i> (and <i>other</i> for paradise papers only). The folder also contains the edges of this graph.

In [ ]:
bahamas_folder = "bahamas/"
panama_folder = "panama/"
paradise_folder = "paradise/"
offshore_folder = "offshore/"

sources_names = ['bahamas', 'panama', 'paradise', 'offshore']

panama_name = panama_folder + "panama_papers"
paradise_name = paradise_folder + "paradise_papers"
offshore_name = offshore_folder + "offshore_leaks"
bahamas_name = bahamas_folder + "bahamas_leaks"

edges_name = ".edges"
nodes_name = ".nodes."

address_name = "address"
intermediary_name = "intermediary"
officer_name = "officer"
entity_name = "entity"
others_name = "other" # Only for paradise paper there is this extra entity

usual_entity_names = [address_name, intermediary_name, officer_name, entity_name]

## Build local storage

We store data in dictionnaries that map each leak source to its content, which is a dictionnary that maps each type of entity to the Dataframe containing its values. For example <b>d_sources["bahamas"]["officer"]</b> is the Dataframe of officers coming from the bahamas leaks.

In [ ]:
def my_read_csv(filename) :
    """ To have same rules when reading data from csv """
    return pd.read_csv(filename, dtype = str)

def build_dict(source_name):
    """
    Create a dictionnary for a certain source_name (among : Panama papers, Paradise papers...)
    that maps to each entity name (among : Officer, Intermediary, Address...)
    the content of the csv from source_name for this entity
    """
    d = {en : my_read_csv(source_name + nodes_name + en + ".csv") for en in usual_entity_names}
    
    if source_name == paradise_name: # Extra "other" entity in paradise papers
        d[others_name] = my_read_csv(source_name + nodes_name + others_name + ".csv")
    
    #Add edges
    d["edges"] = my_read_csv(source_name + edges_name + ".csv")
              
    return d

Build the dictionnary, that maps each source to its content

In [ ]:
d_sources = dict()
d_sources["bahamas"] = build_dict(bahamas_name)
d_sources["panama"] = build_dict(panama_name)
d_sources["paradise"] = build_dict(paradise_name)
d_sources["offshore"] = build_dict(offshore_name)

In [ ]:
d_sources['panama']['entity'].columns

## Getting familiar with the data format

### Define some coloring for printing

Keep the same coloring during the project, it makes data very easily readable once you get familiar with the coloring !

In [ ]:
BOLD = '\033[1m'
BLUE = '\033[94m'
GREEN = '\033[92m'
YELLOW = '\033[93m'
RED = '\033[91m'
END = '\033[0m'

color_dict = dict()
color_dict["bahamas"] = YELLOW
color_dict["paradise"] = GREEN
color_dict["panama"] = RED
color_dict["offshore"] = BLUE

def color(str):
    """
    Returns the str given in the color of the source it is from 
    (the str must contain source name)
    """
    for source in color_dict.keys():
        if source in str:
            return color_dict[source] + str + END 
        
    return BOLD + str + END #Default color is BOLD

for name, _ in color_dict.items():
    print(color(name))
print(color("Unknown source"))

### See what data source misses which column

In [ ]:
for source, dict_data in d_sources.items():
    for source_compare, dict_data_compare in d_sources.items():
        print("\n", color(source_compare), "missing columns from source :", color(source))
        for entity in usual_entity_names:
            missing_columns = []
            for col in dict_data[entity].columns:
                if not col in dict_data_compare[entity].columns:
                    missing_columns.append(col)
            if(len(missing_columns) > 0):
                print("Node type", entity, "misses", len(missing_columns), "columns, namely : ", missing_columns)


We see that <span style="color:orange">bahamas</span> is the most "complete" source, in the sense it is the one that has the biggest number of columns missing in the others. We will therefore use it to explore the content of columns. *'inactivation_date'* and  *'struck_off_date'* columns from entity will then be explored in <span style="color:red">panama</span>

#### Special case : Paradise paper, <i>other</i> node

In [ ]:
d_sources["paradise"]["other"].columns

### SourceID in different sources

We see paradise papers is the only source that has different sourceID

In [ ]:
for source, dict_data in d_sources.items():
    print("\nSource :", color(source))
    for entity in usual_entity_names:
        value_count =  dict_data[entity]["sourceID"].value_counts()
        print("Node :", entity, len(value_count), "different sourceID :")

### Check if node_id is a good index for Nodes

In [ ]:
merged_node_id = pd.Series()

for source, dict_data in d_sources.items():
    merged_node_id_source = pd.Series()
    for entity in usual_entity_names:
        
        merged_node_id_source = merged_node_id_source.append(dict_data[entity]["node_id"], ignore_index = True)
        
        if not dict_data[entity]["node_id"].is_unique:
            print("node_id isn't unique for source", color(source, "node", entity))
                  
    if not merged_node_id_source.is_unique:
        print("node_id isn't unique between nodes from source", color(source))
    
    merged_node_id = merged_node_id.append(merged_node_id_source.drop_duplicates())

if merged_node_id.is_unique:
    print("node_id is unique between unique nodes from all sources")

So for each node type indepently node_id is a good index. Therefore (node_id, node_type) could be a good index (node_type being amond officer, intermediary...)

Now explore nodes with same node_id in offshore

In [ ]:
for i in range(len(usual_entity_names)):
    for j in range(i+1, len(usual_entity_names)):

        left_node = usual_entity_names[i]
        node = usual_entity_names[j]
        print(color(left_node), color(node))
        
        if left_node != node:

            left = d_sources["offshore"][left_node].set_index("node_id")
            right = d_sources["offshore"][node].set_index("node_id")

            intersection = left.join(right, on = "node_id", how = 'inner', \
                                     lsuffix = "_" + left_node,rsuffix = "_" + node)

            if not intersection.empty:
                print("Intersection of", color(left_node), "and", color(node), "count is :")
                print(intersection.count())

So the intersection on offshore is between officer and intermediary nodes. Let's see if they are the same values :

In [ ]:
left = d_sources["offshore"]["officer"].set_index("node_id")
right = d_sources["offshore"]["intermediary"].set_index("node_id")

intersection = left.join(right, on = "node_id", how = 'inner', lsuffix = "_officer",rsuffix = "_interm")

intersection.loc[intersection["name_officer"] != intersection["name_interm"]].empty

Therefore we understand that if someone appears in two different node types, it means it is the same person, but has two roles. This is why in further analysis we will store the pair (node_id, role) as index, because it is unique. We have to add a column to nodes, containing the node type, let's call it label. We saw in the column exploration that bahamas has an equivalent column *labels(n)*, that the other's don't, we'll rename it to *label*

## Keep necessary columns, Shape data to our need

In [ ]:
d_clean = dict()

#maps every node type to the columns to keep
d_columns = dict()
d_columns['address'] = ['country_codes', 'node_id']
d_columns['entity'] = ['node_id','name','jurisdiction','incorporation_date']
d_columns['intermediary'] = ['node_id', 'country_codes','name']
d_columns['officer'] = ['node_id', 'country_codes','name']
d_columns['other'] = ['node_id', 'country_codes','name']


for source, d in d_sources.items():
    
    d_clean[source] = dict()
    
    for node_type in usual_entity_names:
        d_clean[source][node_type] = d[node_type][d_columns[node_type]]
        d_clean[source][node_type]['source'] = source
        d_clean[source][node_type]['type'] = node_type
        d_clean[source][node_type]['node_id'] = d_clean[source][node_type]['node_id'].astype(np.int32)
    
    columns_edges = ['START_ID', 'END_ID', 'TYPE', 'start_date', 'end_date']        
    columns_edges_bahamas = ['node_1', 'node_2', 'rel_type', 'start_date', 'end_date']    
    
    if source == "bahamas": # adapt different column names
        d_clean[source]['edges'] = d_sources[source]['edges'][columns_edges_bahamas]
        d_clean[source]['edges'].columns = columns_edges
        
    else :
        d_clean[source]['edges'] = d_sources[source]['edges'][columns_edges]
    
    d_clean[source]['edges']['source'] = source
    d_clean[source]['edges']["START_ID"] = d_clean[source]['edges']["START_ID"].astype(np.int32)
    d_clean[source]['edges']["END_ID"] = d_clean[source]['edges']["END_ID"].astype(np.int32) 

d_clean['paradise']['other'] = d_sources['paradise']['other'][d_columns['other']]
d_clean["paradise"]['other']['source'] = 'paradise'
d_clean["paradise"]['other']['type'] = 'other'

### Create dictionaries for countries and jurisdictions

These dictionaries map the abrevation of countries to their full name, this way we can drop the longer column

In [ ]:
countries = dict()
jurisdictions = dict()

for s in sources_names:
    for t in usual_entity_names:
        countries.update(dict(zip(d_sources[s][t]['country_codes'], d_sources[s][t]['countries'])))
        if t  == 'entity':
            jurisdictions.update(dict(zip(d_sources[s][t]['jurisdiction'], d_sources[s][t]['jurisdiction_description'])))
            
countries.update(dict(zip(d_sources['paradise']['other']['country_codes'],\
                          d_sources['paradise']['other']['countries'])))

## Create and study *node* dataframe

A pourcentage function to print pourcentages in a nice way

In [ ]:
def pourcentage(n, precision = 2):
    """ To print a pourcentage in a nice way and with a given precision"""
    return color(("%." + str(precision) + "f") % (100.0*n) + "%")

A function to convert string of date to datetime format. There are a LOT of different string formats and we cover most of them. Dates with ambiguity such as 01/03/2001 are treated arbitrarily. (i.e. is this date 1st of March or 3rd of January ?) Indeed the year is generally what matters the most for us.

Years are valid until 2015 at most, and starting in the 1960s according to wikipedia (https://en.wikipedia.org/wiki/Panama_Papers)

When date is clearly an outlier (18/19/2006), it is set to NaN, and printed

In [ ]:
def parse_date(date):
    """ Parsing of the date, read above for more details"""
    if (date==date):
        try:
            formatted = dateutil.parser.parse(date)
            if (formatted.year > 2015 or formatted.year < 1960):
                formatted = 'NaN'
            return formatted 
        except:
            print(date)
            return 'NaN'

### Node types that should contain NaN for each column name

Nodes can have NaN values because of missing data, <b>or</b> because the data doesn't make sense for this node type. You will here find a list of node types for each column, those are node types that are NaN because of this second reason. For example the jurisdiction for an Officer doesn't really make sense at first sight... We will however in the future try to cumpute all the jurisdictions an officer is related to using the edges (and many more)

##### name
- Address

##### jurisdiction and incorporation_date
- Officer
- Other
- Intermediary
- Address

##### country_codes
- Entity


In [ ]:
nodes = pd.DataFrame(columns=['node_id','source','type','name','country_codes', 'jurisdiction', 'incorporation_date'])

for source,_ in d_sources.items():
    for node_type in usual_entity_names:
        nodes = nodes.append(d_clean[source][node_type], sort=False)
#nodes = nodes.append(d_clean['paradise']['other'], sort=False) # Uncomment to consider other nodes

nodes.node_id = pd.to_numeric(nodes.node_id, downcast='integer')

nodes['incorporation_date'] = nodes['incorporation_date'].apply(parse_date)
nodes['incorporation_date'] = pd.to_datetime(nodes['incorporation_date'])

nodes[nodes['country_codes'] == 'XXX']['country_codes'] = None
nodes[nodes['jurisdiction'] == 'XXX']['jurisdiction'] = None

nodes_ = nodes.copy()
nodes = nodes.set_index(['node_id', 'type'])

It looks like there are a lot of unique country_codes... Indeed we notice some nodes have many country codes separated by a ';'

In [ ]:
country_codes = nodes.country_codes.dropna()
number_multi_country = country_codes[country_codes.str.contains(";")].count()

print(pourcentage(number_multi_country/len(country_codes)), "of nodes with a country_code have a country_code with more than one country")

### Study by node type

In [ ]:
nodes.xs('entity', level = 1).describe()

In [ ]:
nodes.xs('officer', level = 1).describe()

In [ ]:
nodes.xs('intermediary', level = 1).describe()

In [ ]:
nodes.xs('address', level = 1).describe()

## Create and study *edges* dataframe

In [ ]:
edges = pd.DataFrame(columns=['START_ID', 'END_ID', 'TYPE', 'start_date', 'end_date','source'])
for source in sources_names:
    edges = edges.append(d_clean[source]['edges'], sort=False)
    
edges.START_ID = pd.to_numeric(edges.START_ID, downcast='integer')
edges.END_ID = pd.to_numeric(edges.END_ID, downcast='integer')

edges['start_date'] = pd.to_datetime(edges['start_date'].apply(parse_date))
edges['end_date'] = pd.to_datetime(edges['end_date'].apply(parse_date))

Printed strings are dates that were not read correctly

In [ ]:
edges.describe()

We see there are <b>13</b> unique kind of edges, they are listed below. In further analysis it will be interesting to study each of them in more depth.

In [ ]:
edges.TYPE.unique()

### Study dates

In [ ]:
number_edges = len(edges)
number_start_date = edges.start_date.notna().sum()
number_end_date = edges.end_date.notna().sum()

In [ ]:
print(pourcentage(number_start_date/number_edges), "of edges have a start date")
print("Among those,", pourcentage(number_end_date/(number_end_date+number_start_date)), "have an end date")
print("The first added relation was the", edges.start_date.min())
print("The last added relation was the", edges.start_date.max())

### Study average and extreme values

In [ ]:
print(edges.START_ID.value_counts().describe(), '\n')
print(edges.END_ID.value_counts().describe())

We see that on average <b>2.98</b> edges start with the same node_id, and <b>2.57</b> end with the same node_id

Max number of connections starting from a given node is <b>36373</b>, and <b>37338</b> ending (another node)

## Study connection between edges and nodes

In [ ]:
id_max_links_start = edges.START_ID.value_counts().idxmax()
max_links_start = edges.START_ID.value_counts().max()
node_max_links_start = nodes.xs(id_max_links_start, level=0)

id_max_links_end = edges.END_ID.value_counts().idxmax()
max_links_end = edges.END_ID.value_counts().max()
node_max_links_end = nodes.xs(id_max_links_end, level=0)

In [ ]:
print("The Node with the most START edges has", color(str(max_links_start)), "links and is :")
print(node_max_links_start[['source', 'name', 'country_codes']])

print("\nThe Node with the most END edges has", color(str(max_links_end)), "links and is :")
print(node_max_links_end[['source', 'country_codes']])

## Useful sets for the following study

In [ ]:
start_nodes = set(edges.START_ID)
end_nodes = set(edges.END_ID)

nodes_address_ids = set(nodes.xs('address', level='type').index.values)
nodes_officer_ids = set(nodes.xs('officer', level='type').index.values)
nodes_entity_ids = set(nodes.xs('entity', level='type').index.values)
nodes_intermediary_ids = set(nodes.xs('intermediary', level='type').index.values)

d_nodes_ids = {
    'address': nodes_address_ids,
    'officer': nodes_officer_ids,
    'entity' : nodes_entity_ids,
    'intermediary' : nodes_intermediary_ids
}

node_ids = frozenset().union(*d_nodes_ids.values())

linked_nodes = start_nodes.union(end_nodes)
isolated_nodes = node_ids.difference(linked_nodes)

## Study nodes that are isolated

In [ ]:
print("There are", color(str(len(isolated_nodes))), "isolated nodes (0 edge connecting them)")

In [ ]:
mask_isolated = nodes.index.get_level_values(0).isin(isolated_nodes)
nodes[mask_isolated].describe()

## Planning

We are planning to infer using the edges:
- all the jurisdictions some nodes such as *officer* have links to, in order to see if see if they are connected to accounts in different jurisdictions.
- for *entity* the country_codes, which would be the country of the people that are connected to this jurisdiction. 

### Update country codes for consistency

**For example 'GBR' and 'UK' are the same but have different code**

In [ ]:
update_code_dict = dict()

wrong_jurisdiction = []

for k, v in jurisdictions.items():
    if len(str(k)) != 3 and not ';' in str(k):
        wrong_jurisdiction.append((k, v))

        
no_matches_codes = []

for k_wrong, v_wrong in wrong_jurisdiction:
    
    match_found = False
    
    #Search in jurisdictions another code with matching description
    for k, v in jurisdictions.items():
        if v == v_wrong and k != k_wrong:
            update_code_dict[k_wrong] = k  #Map old key wrong to corresponding one
            match_found = True
    if not match_found :
        #Search in countries another code with matching description
        for k, v in countries.items():
            if v == v_wrong and k != k_wrong:
                update_code_dict[k_wrong] = k  #Map old key wrong to corresponding one
                match_found = True
    
    if not match_found:
        no_matches_codes.append((k_wrong, v_wrong))

    for k,v in update_code_dict.items():
        if len(k) < len(v):
            update_code_dict[v] = k
            del update_code_dict[k]
print("No match : ", no_matches_codes)

Manually convert the ones with no natural match

In [ ]:
# Method to find countries or jurisdictions with similar descriptions, custom use

descriptions_countries_jurisdictions = list(jurisdictions.values()) + list(countries.values())
for desc in descriptions_countries_jurisdictions:
    if 'U.' in str(desc) and 'Virgin Islands' in str(desc) :
        print(desc)
        break
jurisdications_inv = {v: k for k, v in jurisdictions.items()}
countries_inv = {v: k for k, v in countries.items()}

print(jurisdications_inv.get('U.S. Virgin Islands',""))
print(countries_inv.get('U.S. Virgin Islands',""))

Manually found equivalents

In [ ]:
update_code_dict['US'] = 'USA'
update_code_dict['CAYMN'] = 'CYM'
update_code_dict['STLU'] = 'LCA'
update_code_dict['VI'] = 'VIR'

Apply the updating map to the nodes dataframe

In [ ]:
def map_with_default(x):
    return update_code_dict.get(x, x)

In [ ]:
nodes_['jurisdiction'] = nodes_['jurisdiction'].map(map_with_default, na_action = 'ignore')
nodes = nodes_.set_index(['node_id', 'type'])

# Add all content to edges

In [ ]:
merge_nodes = nodes.reset_index()[['node_id', 'type', 'country_codes', 'jurisdiction']]
merge_nodes.columns = ['id', 'start_type', 'start_country', 'start_jurisdiction']

edges_completed = edges.merge(merge_nodes, how = 'left', left_on='START_ID', right_on='id')

merge_nodes.columns = ['id', 'end_type', 'end_country', 'end_jurisdiction']

edges_completed = edges_completed.merge(merge_nodes, how = 'left', left_on='END_ID', right_on='id')

edges_completed = edges_completed.drop(columns = ['id_x', 'id_y'])

### Useful masks

In [ ]:
# country_code
mask_diff_start_end_address = (edges_completed['start_country'] != edges_completed['end_country'])
mask_country_notna = (edges_completed['start_country'].notna()) & (edges_completed['end_country'].notna())

# jurisdiction
mask_start_loc_notna = (edges_completed['start_country'].notna()) |  (edges_completed['start_jurisdiction'].notna())
mask_end_loc_notna = (edges_completed['end_country'].notna()) |  (edges_completed['end_jurisdiction'].notna())

# intermediary
mask_intermediary_start = (edges_completed.start_type == 'intermediary')
mask_not_intermediary_start = (edges_completed.start_type != 'intermediary')
mask_intermediary_end = (edges_completed.end_type == 'intermediary')
mask_intermediary = mask_intermediary_end | mask_intermediary_start

# officer
mask_officer_start = (edges_completed.start_type == 'officer')

# Relationships

mask_officer_to_entity = mask_officer_start & (edges_completed['end_type'] == 'entity')

# Edges

mask_edge_intemerdiary_of = (edges_completed.TYPE == 'intermediary_of')


#### Masks for similar nodes

In [ ]:
mask_same_name_as = (edges_completed.TYPE == 'same_name_as')
mask_same_company_as = (edges_completed.TYPE == 'same_company_as')
mask_same_id_as = (edges_completed.TYPE == 'same_id_as')
mask_same_as = (edges_completed.TYPE == 'same_as')
mask_same_address_as = (edges_completed.TYPE == 'same_address_as')
mask_same_intermediary_as = (edges_completed.TYPE == 'same_intermediary_as')
mask_similar_company_as = (edges_completed.TYPE == 'similar_company_as') # We looked there are 200 and they're the same
mask_probably_same_officer_as = (edges_completed.TYPE == 'probably_same_officer_as')

mask_same = (mask_same_name_as | mask_same_company_as | mask_same_id_as | mask_same_as | \
            mask_same_address_as | mask_same_intermediary_as | mask_similar_company_as | \
            mask_probably_same_officer_as) 

## Merge nodes connected with "same..." relationships

Some nodes are connected with edges with a name such as same_name_as, same_company_as...
We decided to merge those nodes, by deleting on of them and merging their content together. When conflict on creation date, we always keep the oldest one. We believe those differences in creation date exist because they come from different sources, and the oldest date is the 'first known' date

TODO, edges with same_name_as, same_company_as, same_id_as,same_as... Merge nodes?

In [ ]:
edges_completed.TYPE.value_counts()

In [ ]:
nodes_same_as = edges_completed[mask_same][['START_ID', 'END_ID']]


start_nodes_same = nodes_.loc[nodes_.node_id.isin(nodes_same_as.START_ID)]
end_nodes_same = nodes_.loc[nodes_.node_id.isin(nodes_same_as.END_ID)]

same_nodes_merge = nodes_same_as.merge(start_nodes_same, how='inner', left_on='START_ID', right_on='node_id')\
                                .merge(end_nodes_same, how='inner', left_on='END_ID', right_on='node_id', suffixes=('_start','_end'))\
                                .drop(columns=['START_ID', 'END_ID'])

same_nodes_merge.node_id_start = pd.to_numeric(same_nodes_merge.node_id_start, downcast='integer')
same_nodes_merge.node_id_end = pd.to_numeric(same_nodes_merge.node_id_end, downcast='integer')

same_nodes_merge

We see they are indeed pretty much the same

In [ ]:
same_nodes_diff_type_mask = same_nodes_merge.type_start != same_nodes_merge.type_end
same_nodes_diff_type_count = len(same_nodes_merge[same_nodes_diff_type_mask])

same_nodes_diff_country_mask = (same_nodes_merge.country_codes_start.notna())\
                            & (same_nodes_merge.country_codes_end.notna()) \
                            &(same_nodes_merge.country_codes_start != same_nodes_merge.country_codes_end)
same_nodes_diff_country_count = len(same_nodes_merge[same_nodes_diff_country_mask])

same_nodes_diff_jurisdiction_mask = (same_nodes_merge.jurisdiction_end.notna())\
                            & (same_nodes_merge.jurisdiction_start.notna()) \
                            &(same_nodes_merge.jurisdiction_end != same_nodes_merge.jurisdiction_start)
same_nodes_diff_jurisdiction_count = len(same_nodes_merge[same_nodes_diff_country_mask])

same_nodes_merge = same_nodes_merge[(~same_nodes_diff_type_mask) & \
                                    (~same_nodes_diff_country_mask) & \
                                    (~same_nodes_diff_jurisdiction_mask) ]

print("Node not to merge : ")
print("%d nodes that are similar but have not the same type" % same_nodes_diff_type_count)
print("%d nodes that are similar but have not the same country" % same_nodes_diff_country_count)
print("%d nodes that are similar but have not the same jurisdiction" % same_nodes_diff_jurisdiction_count)

The two other columns that can differ are source, name and the node_id:
- For source we will arbitrarily take the start one, since we don't really care in source analyis in further work.
- For the name we will also take the start one, usually differences are very small (caps, or missing letter...)
- For the node_id we will also take the start one, and erase the end one in nodes. We will moreover replace its occurences in edges with the first one

In [ ]:
same_nodes_merge['merge_date'] = same_nodes_merge[['incorporation_date_start','incorporation_date_end']].min(axis=1)
nodes_replace = same_nodes_merge[['node_id_end', 'node_id_start','type_start', 'source_start', 'name_start',\
                         'country_codes_start', 'jurisdiction_start', 'merge_date']]
nodes_replace.columns = ['old_node_id', 'node_id', 'type', 'source', 'name', 'country_codes', 'jurisdiction', 'incorporation_date']

old_ids_to_replace = set(nodes_replace.old_node_id.unique())

# Remove nodes that are both in old and new
nodes_replace = nodes_replace[~nodes_replace['node_id'].isin(old_ids_to_replace)]

ids_to_update = set(nodes_replace.node_id.unique())

dict_old_new_id = dict()

for index, row in nodes_replace.iterrows():
    dict_old_new_id[row['old_node_id']] = row['node_id']

Delete nodes of old_node_id value, and update the other ones with the computed merge value

In [ ]:
nodes_ = nodes_[~nodes_.node_id.isin(old_ids_to_replace)]
nodes_.update(nodes_replace)
nodes = nodes_.set_index(['node_id', 'type'])

### Remove same* edges from edges_completed

In [ ]:
edges_completed

### Update the edges with the old nodes ids to new ids

In [ ]:
def map_old_new_default(x):
    return dict_old_new_id.get(x, x)

In [ ]:
print(len(edges_completed['START_ID'].unique()))
print(len(edges_completed['START_ID'].map(map_old_new_default).unique()))
print(len(edges_completed['END_ID'].unique()))
print(len(edges_completed['END_ID'].map(map_old_new_default).unique()))

In [ ]:
edges_completed['START_ID'] = edges_completed['START_ID'].map(map_old_new_default)
edges_completed['END_ID'] = edges_completed['END_ID'].map(map_old_new_default)

### Update the edges with the merger value for new ids

In [ ]:
# Update when END_ID is in nodes to replace
nodes_replace.columns = ['old_node_id', 'END_ID', 'end_type', 'source', 'name', 'end_country', 'end_jurisdiction', 'incorporation_date']
edges_completed.update(nodes_replace)

# Update when START_ID is in nodes to replace
nodes_replace.columns = ['old_node_id', 'START_ID', 'start_type', 'source', 'name', 'start_country', 'start_jurisdiction', 'incorporation_date']
edges_completed.update(nodes_replace)

Now some edges are linking nodes to themselves, we notice most of those edges are intermediary_of type, meaning some nodes were the same people, but they were thaught to be intermediary of one another !!

TODO:
ACTUALLY MAYBE NOT : maybe same name as means they are for example from the same company but different persons.....

In [ ]:
edges_completed[edges_completed.START_ID == edges_completed.END_ID].describe(include = 'all')

# 1) How does tax evasion evolves over time? Does the number of offshore societies increases?

### How many entities are created each month?

In [ ]:
entity_dates = nodes[nodes['incorporation_date'].notna()]
entity_dates['incorporation_date'] = entity_dates['incorporation_date'].apply(lambda x: (x.month,x.year))
creation_per_month = entity_dates.groupby('incorporation_date').size()
total_per_month = creation_per_month.cumsum()

In [ ]:
creation_per_month.plot(figsize=(20,10))

### How many entities are there in total each month?

In [ ]:
total_per_month.plot(figsize=(20,10))

### How many entities are created each year?

In [ ]:
entity_dates['incorporation_date'] = entity_dates['incorporation_date'].apply(lambda x: x[1])
creation_per_year = entity_dates.groupby('incorporation_date').size()
total_per_year = creation_per_year.cumsum()

In [ ]:
creation_per_year.plot(figsize=(20,10))

### How many entities in total are there each year?

In [ ]:
total_per_year.plot(figsize=(20,10))

### How many edges are created each month?

In [ ]:
def fill_month_year(df):
    temp_edges = df.to_dict()
    for i in range(1960,2016):
        for j in range(1,13):
            if (j,i) not in temp_edges:
                temp_edges[(j,i)] = 0
    return pd.Series(temp_edges).sort_index()

In [ ]:
edges_date = edges_completed[edges_completed['start_date'].notna()]
edges_date['start_date'] = edges_date['start_date'].apply(lambda x: (x.month,x.year))
start_date  = edges_date.groupby('start_date').size()
start_date = fill_month_year(start_date)
start_date.plot(figsize=(20,10))

### How many edges are deleted each month?

In [ ]:
edges_date = edges_completed[edges_completed['end_date'].notna()]
edges_date['end_date'] = edges_date['end_date'].apply(lambda x: (x.month,x.year))
end_date = edges_date.groupby('end_date').size()
end_date = fill_month_year(end_date)
end_date.plot(figsize=(20,10))

### How many edges in total are there each month?

In [ ]:
creation_deletion = start_date.subtract(end_date)
edges_month_sum = creation_deletion.cumsum()

In [ ]:
edges_month_sum.plot(figsize=(20,10))

### How many edges are created each year?

In [ ]:
def fill_year(df):
    temp_edges = df.to_dict()
    for i in range(1960,2016):
        if i not in temp_edges:
            temp_edges[i] = 0
    return pd.Series(temp_edges).sort_index()

In [ ]:
edges_date = edges_completed[edges_completed['start_date'].notna()]
edges_date['start_date'] = edges_date['start_date'].apply(lambda x: x.year)
start_date  = edges_date.groupby('start_date').size()
start_date.plot(figsize=(20,10))

### How many edges are deleted each year?

In [ ]:
edges_date = edges_completed[edges_completed['end_date'].notna()]
edges_date['end_date'] = edges_date['end_date'].apply(lambda x: x.year)
end_date = edges_date.groupby('end_date').size()
end_date = fill_year(end_date)
end_date.plot(figsize=(20,10))

### How many edges in total are there each year?

In [ ]:
creation_deletion = start_date.subtract(end_date)
edges_year_sum = creation_deletion.cumsum()

In [ ]:
edges_year_sum.plot(figsize=(20,10))

# 2) Do people that are related to one offshore account tend to be related to many more? In other words, is it more likely to have another account once you already have one, than it is to have at least one account?

In [ ]:
def histedges_equalN(x, nbin):
    """
        To build bins of equal height (the length will vary)
    """
    npt = len(x)
    return np.interp(np.linspace(0, npt, nbin + 1),
                     np.arange(npt),
                     np.sort(x))

In [ ]:
edges_officer_to_entity = edges_completed[mask_officer_to_entity]
entities_per_officer = edges_officer_to_entity['START_ID'].value_counts()

plt.xlim(1, 15)
plt.ylim(0.7, 1)
#plt.xscale('log')
plt.hist(entities_per_officer, bins=[1, 2, 10, 100, 1000, 10000, 100000], cumulative = True, density=True)

In [ ]:
total_officers = len(entities_per_officer)

pourcentage_one_account = len(entities_per_officer[entities_per_officer == 1]) / total_officers * 100
pourcentage_two_account = len(entities_per_officer[entities_per_officer <= 2]) / total_officers * 100
pourcentage_ten_account = len(entities_per_officer[entities_per_officer <= 10]) / total_officers * 100


print(entities_per_officer.mean())
print('%.2f%% of officer are related to only one account' % pourcentage_one_account)
print('%.2f%% of officer are related to two or less account' % pourcentage_two_account)
print('%.2f%% of officer are related to ten or less account' % pourcentage_ten_account)

# 3) Do many people share one offshore society or do they tend to have their own?

In [ ]:
nodes_entities = nodes_[nodes_.type == 'entity'].node_id.to_frame()

entities_connections = nodes_entities.merge(edges_completed, how = 'left', left_on='node_id', right_on='END_ID')

In [ ]:
connections_per_entity = entities_connections.node_id.value_counts()

entities_connections.start_type.value_counts()

In [ ]:
plt.hist(connections_per_entity.values, bins = 100)
plt.ylim(0, 1100)
plt.plot()

# 4)  *(OLD) How many intermediate societies are there between people and their offshore company?*

## Chgmt de question le mettre dans le README

# 4) Is there a type of offshore society that uses intermediaries rather than  officer relationship

What is an intermediary ??

In [ ]:
map_columns = {'END_ID': 'entity' , 'start_type': 'type_connected' , 'START_ID':'count'}

count_type_entities = entities_connections[['END_ID', 'start_type', 'START_ID']]\
                                .rename(columns=map_columns)\
                                .groupby(['entity','type_connected']).count()

count_type_entities =count_type_entities.join(count_type_entities.reset_index() \
                                              .drop(columns='type_connected') \
                                              .rename(columns={'count':'total'})\
                                              .groupby('entity').sum(), how='left')

count_type_entities['proportion'] = count_type_entities['count']/count_type_entities['total']

TODO now : study for different entity types (size [1] ; [2-10] ; ...)

In [ ]:
count_type_entities[count_type_entities.total > 1]

# 5) Is there a correlation between the location of the people and the location of their offshore society?

## Flatten country codes with multiple countries separated by a coma

In [ ]:
edges_diff_start_end_address = edges_completed[mask_country_notna & mask_diff_start_end_address]\
.end_type.value_counts()

In [ ]:
edges_q5 = edges_completed[mask_officer_to_entity & mask_start_loc_notna & mask_end_loc_notna][['start_country', 'end_jurisdiction']]
edges_q5['sum'] = 1
edges_q5.start_country = edges_q5.start_country.str.split(';')

In [ ]:
flattened_country = []

for index, row in edges_q5[(edges_q5.start_country.map(len) > 1)].iterrows():
    for country in row['start_country']:
        modified_row = row.copy()
        modified_row['start_country'] = country + '_GRP'
        flattened_country.append(modified_row)

edges_q5_flattened = edges_q5[edges_q5['start_country'].map(len) == 1]
edges_q5_flattened.start_country = edges_q5_flattened.start_country.apply(lambda arr: arr[0])
edges_q5_flattened = edges_q5_flattened.append(flattened_country)
      
print(len(edges_q5))
print(len(edges_q5_flattened))

In [ ]:
q5_distrib = edges_q5_flattened.groupby(['start_country', 'end_jurisdiction']).sum()

## Q5 Pie printing

In [ ]:
threshold_pourcentage = 0.05

def my_autopct(pct):
    """
    Only print pct if it is more than threshold
    """
    return ('%1.1f%%' % pct) if pct > 100*threshold_pourcentage else ''

def my_labels(serie):
    """
    Label is the jurisdiction name except if it's lower than threshold then it's ''
    """
    total = serie.sum().values[0]
    return [jurisdictions[row[0]] if row[1].values/total > 0.05 else '' for row in serie.iterrows()]

In [ ]:
one_pie = q5_distrib.xs('USA', level='start_country').sort_values('sum', ascending = False)
labels = my_labels(one_pie)

plt.pie(one_pie, radius = 2, labels=labels, autopct=my_autopct, startangle=90, shadow = True)
plt.show()

## Q5 Scatter

In [ ]:
q5_distrib_country = q5_distrib.groupby('start_country').sum()
q5_distrib_country = q5_distrib.join(q5_distrib_country, how = 'left', on = 'start_country', rsuffix = '_country')
q5_distrib_country['pourcentage'] = q5_distrib_country['sum']/q5_distrib_country['sum_country']
q5_distrib_country.groupby('start_country').count()

In [ ]:
threshold_scatter_pourc = 0.1
threshold_scatter_countries = 10000

threshold_mask = (q5_distrib_country['pourcentage'] > threshold_scatter_pourc) \
& (q5_distrib_country['sum_country'] > threshold_scatter_countries)

q5_thresholded = q5_distrib_country[threshold_mask]

countries_x_scatter =  [val[0] for val in q5_thresholded.index.values]
jurisdictions_y_scatter =  [jurisdictions[val[1]] for val in q5_thresholded.index.values]
scalars_scatter = q5_thresholded['pourcentage'].values * 100

plt.scatter(countries_x_scatter, jurisdictions_y_scatter, scalars_scatter)
plt.show()

We can clearly see Malta and the British Virgin Islands as constant prefered choice whatever the country of origin of the officer.

### UP cell : now plot distribution for each country and see if there is always a leader, what is the variance, %of total for top1, BUILD MAP Where you can click on a country and see the distribution

In [ ]:
edges_completed[(edges_completed['start_jurisdiction'] != edges_completed['end_jurisdiction']) \
               & (edges_completed['start_jurisdiction'].notna()) & ((edges_completed['end_jurisdiction'].notna()))]\
.TYPE.value_counts()

## Create a dataframe of connected components ??

Study the type of START nodes and of END nodes to see if there is a logic (entity always START node etc...)

Should we consider the ones not leading to entity as outliers and withdraw them ?

## TODO : column link is more precise (shareholder of) just not there in bahamas
## Should be added with NaN if not there

## Analysis of start/end nodes

In [ ]:
mask_start_node = nodes.index.get_level_values(0).isin(start_nodes)
mask_end_node = nodes.index.get_level_values(0).isin(end_nodes)

start_nodes_type = nodes[mask_start_node].index.get_level_values("type").tolist()
end_nodes_type = nodes[mask_end_node].index.get_level_values("type").tolist()

In [ ]:
pd.Series(start_nodes_type).value_counts()

So Address is (almost) never an end node (makes sense condsidering the name of edge is "address of")

In [ ]:
pd.Series(end_nodes_type).value_counts()

So intermediary is (almost) never an end node

Entity is the only node type to have jurisdiction, we need to link every entity with the other nodes that do contain a country_code. Let's see first if connected components always have the one country_code, and if not quantify it.

In [ ]:
edges_officer_to_entity[edges_officer_to_entity['TYPE'] == 'officer_of']

Use slicers to access a set of values on index